In [4]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict

In [6]:
# Load and chunk contents of the blog
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

In [11]:
from rich import print
print(docs)

[
    Document(
        metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'},
        page_content='\n\n      LLM Powered Autonomous Agents\n    \nDate: June 23, 2023  |  Estimated Reading 
Time: 31 min  |  Author: Lilian Weng\n\n\nBuilding agents with LLM (large language model) as its core controller is
a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring 
examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it 
can be framed as a powerful general problem solver.\nAgent System Overview#\nIn a LLM-powered autonomous agent 
system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and 
decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of 
complex tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over past actions, 
learn from mistakes and refine them for future steps, thereby improving the quality of final 
results.\n\n\nMemory\n\nShort-term memory: I would consider all the in-context learning (See Prompt Engineering) as
utilizing short-term memory of the model to learn.\nLong-term memory: This provides the agent with the capability 
to retain and recall (infinite) information over extended periods, often by leveraging an external vector store and
fast retrieval.\n\n\nTool use\n\nThe agent learns to call external APIs for extra information that is missing from 
the model weights (often hard to change after pre-training), including current information, code execution 
capability, access to proprietary information sources and more.\n\n\n\n\nFig. 1. Overview of a LLM-powered 
autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs 
to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a 
standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think 
step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT 
transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking 
process.\nTree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each 
step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, 
creating a tree structure. The search process can be BFS (breadth-first search) or DFS (depth-first search) with 
each state evaluated by a classifier (via a prompt) or majority vote.\nTask decomposition can be done (1) by LLM 
with simple prompting like "Steps for XYZ.\\n1.", "What are the subgoals for achieving XYZ?", (2) by using 
task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.\nAnother 
quite distinct approach, LLM+P (Liu et al. 2023), involves relying on an external classical planner to do 
long-horizon planning. This approach utilizes the Planning Domain Definition Language (PDDL) as an intermediate 
interface to describe the planning problem. In this process, LLM (1) translates the problem into “Problem PDDL”, 
then (2) requests a classical planner to generate a PDDL plan based on an existing “Domain PDDL”, and finally (3) 
translates the PDDL plan back into natural language. Essentially, the planning step is outsourced to an external 
tool, assuming the availability of domain-specific PDDL and a suitable planner which is common in certain robotic 
setups but not in many other domains.\nSelf-Reflection#\nSelf-reflection is a vital aspect that allows autonomous 
agents to improve iteratively by refining past action decisions and correcting previous mistakes. It plays a 
crucial role in real-world tasks where trial and error are inevitabl

In [12]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)
print(all_splits)

[
    Document(
        metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'},
        page_content='LLM Powered Autonomous Agents\n    \nDate: June 23, 2023  |  Estimated Reading Time: 31 min  
|  Author: Lilian Weng\n\n\nBuilding agents with LLM (large language model) as its core controller is a cool 
concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. 
The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be 
framed as a powerful general problem solver.\nAgent System Overview#\nIn a LLM-powered autonomous agent system, LLM
functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: 
The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex 
tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn 
from mistakes and refine them for future steps, thereby improving the quality of final results.\n\n\nMemory'
    ),
    Document(
        metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'},
        page_content='Memory\n\nShort-term memory: I would consider all the in-context learning (See Prompt 
Engineering) as utilizing short-term memory of the model to learn.\nLong-term memory: This provides the agent with 
the capability to retain and recall (infinite) information over extended periods, often by leveraging an external 
vector store and fast retrieval.\n\n\nTool use\n\nThe agent learns to call external APIs for extra information that
is missing from the model weights (often hard to change after pre-training), including current information, code 
execution capability, access to proprietary information sources and more.'
    ),
    Document(
        metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'},
        page_content='Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA 
complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask 
Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing 
model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time 
computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple 
manageable tasks and shed lights into an interpretation of the model’s thinking process.'
    ),
    Document(
        metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'},
        page_content='Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities 
at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step,
creating a tree structure. The search process can be BFS (breadth-first search) or DFS (depth-first search) with 
each state evaluated by a classifier (via a prompt) or majority vote.\nTask decomposition can be done (1) by LLM 
with simple prompting like "Steps for XYZ.\\n1.", "What are the subgoals for achieving XYZ?", (2) by using 
task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.'
    ),
    Document(
        metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'},
        page_content='Another quite distinct approach, LLM+P (Liu et al. 2023), involves relying on an external 
classical planner to do long-horizon planning. This approach utilizes the Planning Domain Definition Language 
(PDDL) as an intermediate interface to describe the planning problem. In this process, LLM (1) translates the 
problem into “Problem PDDL”, then (2) requests a classical planner to generate a PDDL plan based on an existing 
“Domain PDDL”, and finally (3) translates the PDDL pla

In [21]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

NameError: name 'embeddings' is not defined